# **License Plate Detection using Detectron2, PyTorch, Colab, Roboflow and Tesseract**

### **Objective**
This notebook shows an end to end pipeline for detecting license plates from images using Detectron2, PyTorch, and Tesseract.  It uses Google''c Colab for using GPUs, and Roboflow for generating Coco compatible image dataset.  It also caches weights for a trained model for reuse during inference operations.

### **Tools and references**
Here are references to the tools used:

Cars dataset:  https://www.kaggle.com/andrewmvd/car-plate-detection

Detectron2:  https://ai.facebook.com/tools/detectron2/

Tesseract:  https://pypi.org/project/pytesseract/

Google Colab:  https://research.google.com/colaboratory/faq.html

Roboflow:  https://roboflow.com/

Coco data format:  https://opencv.org/introduction-to-the-coco-dataset/#:~:text=COCO%20stores%20data%20in%20a,level%20description%20of%20the%20dataset

A related reference: https://towardsdatascience.com/object-detection-in-6-steps-using-detectron2-705b92575578

Image denoising:  https://stackoverflow.com/questions/37745519/use-pytesseract-ocr-to-recognize-text-from-an-image

Computer Vision references:  https://courses.opencv.org/dashboard


Links to my dataset:  



### **Approach**

This notebook uses Google Colab for access to GPUs.  
The full pipeline uses the following steps:

1.  Mount Colab
2.  Install Detectron2 dependencies
3.  Install Detectron2
4.  Import packages
5.  Register Dataset:
    The dataset needs to be registered with detectron2 to be used.
    The dataset format should be compatible with Coco format (see 
    references).  We used Roboflow to convert our data to Coco format 
    (references)
6.  Visualize training data.  Visualizing data helps in ensuring we use the right dataset for training which can be computationally expensive.
7.  Create CocoTrainer class.
8.  Train the model.
9.  Inspect training curves in tensorboard.
10.  Define predictor and evaluator objects.
11.  Save weights for the trained model.
12.  List output files.
13.  Assign proper weights to configuration object.
14.  Denoise image.
15.  Infer and visualize predictions.  

### **Note** 
This notebook describes an end to end approach to detect license plates from car images.  I will update these notes with approaches which enhance the accuracy and execution time as I get additional information.

###  **Summary**
We are usng detectron2 to localize the license plate bounding box of a car.  This localized license plate image is fed to an OCR tool like Tesseract for licens plate detection.  We could improve the accuracy of detection by denoising the image before feeding it to the OCR tool.  Another approach would be to use third party APIs for OCR function like:

https://platerecognizer.com/alpr-results





###1.  Mount colab

In [ ]:
# Note:  Change runtime type to a GPU
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

print('done')


###2. **Install Detectron2 Dependencies**

In [ ]:
#Note:  use cu101 because colab has CUDA 10.1

!pip install -U torch==1.5 torchvision==0.6 -f https://download.pytorch.org/whl/cu101/torch_stable.html 
!pip install cython pyyaml==5.1
!pip install -U 'git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI'
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())
!gcc --version
import random
#------------------------------
# run parameters
is_train = 0 # set to 1 for training model for a new dataset, or 0 to use weights from a previous training run
model_path = '/content/drive/MyDrive/content2/datasetsAndJNs/text_detection/data/cached_model'
max_iter = 1000
score_threshold_test = 0.75
#-----------------------------------------------
train_jsons  =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/train/_annotations.coco.json"
train_images =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/train"

valid_jsons  =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/valid/_annotations.coco.json"
valid_images =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/valid"

test_jsons  =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test/_annotations.coco.json"
test_images =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test"

test_lp_jsons  =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test_lp/_annotations.coco.json"
test_lp_images =  "/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test_lp"
##-----------------------------------------------------------
# train_jsons  =  "path to your train json file"
# train_images =  "path to your train images'

# valid_jsons  =  "path to your valid json file"
# valid_images =  "path to your valid images"

# test_jsons  =  "path to your test json file"
# test_images =  "path to your test images"

# test_lp_jsons  =  "path to your test_lp json file"
# test_lp_images =  "path to your test_lp images"
#------------------------------------------------------

dsv = str(random.randint(0,1000000))  ## dataset version

print('done')

###3.  i**nstall Detectron2**

In [ ]:
!pip install detectron2==0.1.3 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cu101/torch1.5/index.html

print('done')

###4.  **imports**

In [ ]:
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import os
import cv2
import random
from google.colab.patches import cv2_imshow

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog
from detectron2.data.catalog import DatasetCatalog

print('done')

###*5*. **register datasets**

In [ ]:
from detectron2.data.datasets import register_coco_instances
##-------------------------------------------------------------
# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/train/_annotations.coco.json

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/train

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/valid/_annotations.coco.json

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/valid

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test/_annotations.coco.json

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test_lp/_annotations.coco.json

# !ls /content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco/test_lp

register_coco_instances("my_dataset_train" + dsv, {}, train_jsons, train_images)

register_coco_instances("my_dataset_valid" + dsv, {}, valid_jsons, valid_images)

register_coco_instances("my_dataset_test" + dsv, {}, test_jsons, test_images)

register_coco_instances("my_dataset_test_lp" + dsv, {}, test_jsons, test_lp_images)

print('done')


###6.  **visualize training data**

In [ ]:
my_dataset_train_metadata = MetadataCatalog.get("my_dataset_train" + dsv)
dataset_dicts = DatasetCatalog.get("my_dataset_train" + dsv)

import random
from detectron2.utils.visualizer import Visualizer

for d in random.sample(dataset_dicts, 3):
    fname = d["file_name"]
    img = cv2.imread(fname)
    visualizer = Visualizer(img[:, :, ::-1], metadata=my_dataset_train_metadata, scale=0.5)
    vis = visualizer.draw_dataset_dict(d)
    cv2_imshow(vis.get_image()[:, :, ::-1])

###7. **train Custom Detectron2 Detector**

In [ ]:
# We are importing our own Trainer Module here to use the COCO validation evaluation during training. Otherwise no validation eval occurs.

from detectron2.engine import DefaultTrainer
from detectron2.evaluation import COCOEvaluator

class CocoTrainer(DefaultTrainer):

  @classmethod
  def build_evaluator(cls, cfg, dataset_name, output_folder=None):
    coco_eval = '/content/drive/MyDrive/content2/datasetsAndJNs/cars_kaggle/data_coco'
    if output_folder is None:
        os.makedirs(coco_eval, exist_ok=True)
        output_folder = coco_eval

    return COCOEvaluator(dataset_name, cfg, False, output_folder)

print('done')

###8. **train**

In [ ]:
from detectron2.engine import DefaultTrainer

cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml"))

cfg.DATASETS.TRAIN = ("my_dataset_train" + dsv,)
cfg.DATASETS.TEST = ("my_dataset_valid" + dsv,) ## datasets.test should be valid dataset

cfg.DATALOADER.NUM_WORKERS = 2

cfg.MODEL.WEIGHTS = model_zoo.get_checkpoint_url("COCO-Detection/faster_rcnn_X_101_32x8d_FPN_3x.yaml")  # Let training initialize from model zoo

cfg.SOLVER.IMS_PER_BATCH = 2  # This is the real "batch size" commonly known to deep learning people
cfg.SOLVER.BASE_LR = 0.00025  # pick a good LR
cfg.SOLVER.MAX_ITER = max_iter    # 300 iterations seems good enough for this toy dataset; you will need to train longer for a practical dataset
cfg.SOLVER.STEPS = []        # do not decay learning rate
cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128   # The "RoIHead batch size". 128 is faster, and good enough for this toy dataset (default: 512)
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2  # only has one class (ballon). (see https://detectron2.readthedocs.io/tutorials/datasets.html#update-the-config-for-new-datasets)
# NOTE: this config means the number of classes, but a few popular unofficial tutorials incorrect uses num_classes+1 here.

os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = DefaultTrainer(cfg) 
trainer.resume_or_load(resume=False)
if(is_train==1):
  trainer.train()
else:
  trainer = torch.load(model_path + '/model_final.pth')

print('done')

###9.  **look at training curves in tensorboard**

In [ ]:
%load_ext tensorboard
%tensorboard --logdir output



```
# This is formatted as code
```

###10.  **Define predictor and evaluator objects**

In [ ]:
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader
from detectron2.evaluation import COCOEvaluator, inference_on_dataset

weights_path = cfg.OUTPUT_DIR
if(is_train!=1):
  weights_path = model_path

cfg.MODEL.WEIGHTS = os.path.join(weights_path, "model_final.pth")

cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = score_threshold_test
predictor = DefaultPredictor(cfg)
evaluator = COCOEvaluator("my_dataset_test" + dsv, cfg, False, output_dir="./output/")
val_loader = build_detection_test_loader(cfg, "my_dataset_test" + dsv)
if(is_train==1):
  inference_on_dataset(trainer.model, val_loader, evaluator)

print('done')

###11,  **save weights**

In [ ]:
from detectron2.checkpoint import DetectionCheckpointer, Checkpointer

if (is_train==1):
  checkpointer = DetectionCheckpointer(trainer.model, save_dir=model_path)
  checkpointer.save("model_final")  
#----------------------
print('done')


###12. **List output files**



In [ ]:
%ls ./output/

###13.  **Assign proper weights to configuration object**


In [ ]:
cfg.DATASETS.TEST = ("my_dataset_test" + dsv, )

test_metadata = MetadataCatalog.get("my_dataset_test" + dsv)

print('done')

###14.  **Denoise input image**

In [ ]:
def denoise(image):
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  blur = cv2.GaussianBlur(gray, (3,3), 0)
  thresh = cv2.threshold(blur, 150, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

  # Morph open to remove noise and invert image
  kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
  opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
  invert = 255 - opening
  return invert

###15.  **infer and visualize perdctions**

In [ ]:
from detectron2.utils.visualizer import ColorMode
import glob

#-------------
!sudo apt-get install tesseract-ocr
!pip install pytesseract==0.3.9
import pytesseract
from pytesseract import Output
from PIL import Image
import cv2
#------------
bboxes2 = []
ct_cars_detected = 0
ct_images = 0
cars_undetected = []
cars_detected = []
cars_lp_detected = []
cars_all = []
for imageName in glob.glob(test_lp_images + '/*jpg'):
  ct_images = ct_images + 1
  cars_all.append(imageName)
  im = cv2.imread(imageName)
  outputs = predictor(im)

  output_pred_boxes = outputs["instances"].pred_boxes

  for i in output_pred_boxes.__iter__():
    bb = i.cpu().numpy()
    flag = np.any(bb)
    if(flag):
        bboxes2.append((imageName, bb))
        print('bounding box', bb)
        ct_cars_detected = ct_cars_detected + 1
        bboxes2.append((imageName, bb))
        #-------------------------------------
        y1 = int(bb[0])
        x1 = int(bb[1])
        y2 = int(bb[2])
        x2 = int(bb[3])

        print('x1,y1,x2, y2', x1, y1, x2, y2)
        license_plate = im[x1:x2, y1:y2]
        print('image')
        cv2_imshow(im)
        print('license_plate')
        cv2_imshow(license_plate)
        im_denoise = denoise(im)

        cv2.waitKey(0)
        #------------------
        lpn = pytesseract.image_to_string(license_plate)
        print('license plate number: ', lpn)

        # lpn_denoise = pytesseract.image_to_string(im_denoise)
        # print('license_plate_number_denoise: ', lpn_denoise)

print('done') 